In [70]:
import warnings
warnings.filterwarnings("ignore")

In [107]:
import numpy as np
import pandas as pd  
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, root_mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.compose import make_column_transformer 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import KFold, GridSearchCV

In [72]:
cars_df = pd.read_csv(r"C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Data Sets\Used Card Price\train.csv", index_col=0)
cars_df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
id,,,,,,,,,,,,
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [73]:
cars_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 188533 entries, 0 to 188532
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   brand         188533 non-null  object
 1   model         188533 non-null  object
 2   model_year    188533 non-null  int64 
 3   milage        188533 non-null  int64 
 4   fuel_type     183450 non-null  object
 5   engine        188533 non-null  object
 6   transmission  188533 non-null  object
 7   ext_col       188533 non-null  object
 8   int_col       188533 non-null  object
 9   accident      186081 non-null  object
 10  clean_title   167114 non-null  object
 11  price         188533 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 18.7+ MB


In [74]:
cars_df.isnull().sum()

brand               0
model               0
model_year          0
milage              0
fuel_type        5083
engine              0
transmission        0
ext_col             0
int_col             0
accident         2452
clean_title     21419
price               0
dtype: int64

In [75]:
print(cars_df['fuel_type'].value_counts())

fuel_type
Gasoline          165940
Hybrid              6832
E85 Flex Fuel       5406
Diesel              3955
–                    781
Plug-In Hybrid       521
not supported         15
Name: count, dtype: int64


In [76]:
print(cars_df['accident'].value_counts())

accident
None reported                             144514
At least 1 accident or damage reported     41567
Name: count, dtype: int64


In [77]:
print(len(cars_df['engine'].unique()))

1117


In [78]:
print(cars_df['transmission'].value_counts())

transmission
A/T                                                   49904
8-Speed A/T                                           20645
Transmission w/Dual Shift Mode                        19255
6-Speed A/T                                           18044
6-Speed M/T                                           11998
7-Speed A/T                                           11124
Automatic                                             10691
8-Speed Automatic                                      8431
10-Speed A/T                                           8044
9-Speed A/T                                            3866
5-Speed A/T                                            3217
10-Speed Automatic                                     3164
6-Speed Automatic                                      2799
4-Speed A/T                                            2546
5-Speed M/T                                            2409
9-Speed Automatic                                      2325
CVT Transmission           

In [79]:
print(len(cars_df['clean_title']) - cars_df['clean_title'].value_counts()['Yes'])
print(cars_df['clean_title'].value_counts())

21419
clean_title
Yes    167114
Name: count, dtype: int64


In [80]:
X_train = cars_df.drop('price', axis=1)
y_train = cars_df['price']

In [99]:
y_test = pd.read_csv(r'C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Data Sets\Used Card Price\sample_submission.csv')

In [81]:
ohe = OneHotEncoder(
    sparse_output=False, 
    drop='first',
    handle_unknown='infrequent_if_exist',
    min_frequency=3000
).set_output(transform='pandas')



ct_ohe = make_column_transformer(
    ('passthrough', make_column_selector(dtype_exclude=object)),
    (ohe, make_column_selector(dtype_include=object)),
    verbose_feature_names_out=False
).set_output(transform='pandas')

In [82]:
X_test = pd.read_csv(r"C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Data Sets\Used Card Price\test.csv", index_col=0)

In [83]:
X_train_ohe = ct_ohe.fit_transform(X_train)
X_test_ohe = ct_ohe.transform(X_test)

In [84]:
X_train_ohe.isnull().sum().sum()

0

In [108]:
lr = LinearRegression()
lr.fit(X_train_ohe, y_train)
y_pred = lr.predict(X_test_ohe)

In [115]:
y_pred[y_pred< 150] = 150

In [116]:
print('R2: ',r2_score(y_test['price'], y_pred))
print('MAE: ',mean_absolute_error(y_test['price'], y_pred))
print('MSE: ',mean_squared_error(y_test['price'], y_pred))
print('RMSE: ',root_mean_squared_error(y_test['price'], y_pred))

R2:  -1.1318248362524035e+30
MAE:  19457.81823605122
MSE:  539264771.3399001
RMSE:  23222.075086862933


In [119]:
en = ElasticNet(alpha=0.5, l1_ratio=0.5)
en.fit(X_train_ohe, y_train)
y_pred = en.predict(X_test_ohe)

In [117]:
y_pred[y_pred< 150] = 150

In [118]:
print('R2: ',r2_score(y_test['price'], y_pred))
print('MAE: ',mean_absolute_error(y_test['price'], y_pred))
print('MSE: ',mean_squared_error(y_test['price'], y_pred))
print('RMSE: ',root_mean_squared_error(y_test['price'], y_pred))

R2:  -1.1318248362524035e+30
MAE:  19457.81823605122
MSE:  539264771.3399001
RMSE:  23222.075086862933


In [122]:
sanple_out = y_test.copy()
sanple_out['price'] = y_pred
sanple_out.to_csv('UsedCarsKnnRegression.csv', index=False)